# Acoustic Features

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
acoustics = pd.read_csv("acoustic_features.csv")
billboard = pd.read_csv("albums.csv")
pitchfork = pd.read_csv("reviews.csv")
billboard_pitchfork = pd.read_pickle("./grouped_billboard.pkl")

In [3]:
acoustics

,index,id,song,album,artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,album_id,date
0,0,0Veyvc3n9AcLSoK3r1dA12,Voices In My Head,Hoodie SZN,A Boogie Wit da Hoodie,0.055500,0.754,142301.0,0.663,0.000000,6.0,0.1010,-6.311,0.0,0.4270,90.195,4.0,0.2070,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
1,1,77JzXZonNumWsuXKy9vr3U,Beasty,Hoodie SZN,A Boogie Wit da Hoodie,0.292000,0.860,152829.0,0.418,0.000000,7.0,0.1060,-9.061,0.0,0.1580,126.023,4.0,0.3740,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
2,2,18yllZD0TdF7ykcREib8Z1,I Did It,Hoodie SZN,A Boogie Wit da Hoodie,0.153000,0.718,215305.0,0.454,0.000046,8.0,0.1160,-9.012,1.0,0.1270,89.483,4.0,0.1960,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
3,3,1wJRveJZLSb1rjhnUHQiv6,Swervin (feat. 6ix9ine),Hoodie SZN,A Boogie Wit da Hoodie,0.015300,0.581,189487.0,0.662,0.000000,9.0,0.1110,-5.239,1.0,0.3030,93.023,4.0,0.4340,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
4,4,0jAfdqv18goRTUxm3ilRjb,Startender (feat. Offset and Tyga),Hoodie SZN,A Boogie Wit da Hoodie,0.023500,0.736,192779.0,0.622,0.000000,6.0,0.1510,-4.653,0.0,0.1330,191.971,4.0,0.5060,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
5,5,3L19besdNQzd342qL78xqm,Demons and Angels (feat. Juice WRLD),Hoodie SZN,A Boogie Wit da Hoodie,0.011400,0.810,214593.0,0.550,0.000006,2.0,0.1070,-7.460,0.0,0.1700,76.503,4.0,0.1830,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
6,6,1Gno5IBUHh6vUKp4nxu2Yd,Love Drugs and Sex,Hoodie SZN,A Boogie Wit da Hoodie,0.550000,0.762,157863.0,0.553,0.000000,5.0,0.1150,-7.207,0.0,0.1120,119.973,4.0,0.7730,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
7,7,2A4vCndSkZ3jd5nfh3a8gc,Skeezers,Hoodie SZN,A Boogie Wit da Hoodie,0.558000,0.792,198054.0,0.484,0.000000,7.0,0.1340,-7.801,0.0,0.1550,140.019,4.0,0.3530,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
8,8,1srm5YnZiu7Ksnd2UTTmmk,Savage,Hoodie SZN,A Boogie Wit da Hoodie,0.188000,0.780,169046.0,0.402,0.000000,5.0,0.0597,-9.255,0.0,0.1490,160.046,4.0,0.4830,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21
9,9,3UhRsQVrELOATEiY7E9X2T,Come Closer (feat. Queen Naija),Hoodie SZN,A Boogie Wit da Hoodie,0.220000,0.741,156512.0,0.556,0.000000,7.0,0.0981,-6.647,0.0,0.0947,140.056,4.0,0.6160,3r5hf3Cj3EMh1C2saQ8jyt,2018-12-21


## TODO

